# Loading more information about the hotels

## Extract data from the metadata table in the sqlite database

In [1]:
# Dependencies
import pandas as pd
import numpy as np
import sqlite3
from IPython.core.display import clear_output

import nltk
nltk.download("punkt")
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/rochiecuevas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Create a connection to the database
conn = sqlite3.connect("Data/Hotels.db")

# Load the database table into a pandas dataframe
metadata = pd.read_sql_query("select * from metadata;", conn)
conn.close()

# Preview the dataframe
metadata.head()

,index,name,categories,primaryCategories,address,city,province,latitude,longitude,websites
0,0,Rancho Valencia Resort Spa,"Hotels,Hotels and motels,Hotel and motel reser...",Accommodation & Food Services,5921 Valencia Cir,Rancho Santa Fe,CA,32.990959,-117.186136,http://www.ranchovalencia.com
1,3,Aloft Arundel Mills,"Hotels,Hotels and motels,Travel agencies and b...",Accommodation & Food Services,7520 Teague Rd,Hanover,MD,39.155929,-76.716341,http://www.starwoodhotels.com/alofthotels/prop...
2,9,Hampton Inn Suites PortlandVancouver,"Hotels,Hotels and motels,Hotel and motel reser...",Accommodation & Food Services,315 SE Olympia Dr,Vancouver,WA,45.619212,-122.525196,http://hamptoninn3.hilton.com/en/hotels/washin...
3,15,Hotel Phillips,"Hotels,Caterers,Hotels and motels,Hotel,Restau...",Accommodation & Food Services,106 W 12th St,Kansas City,MO,39.100119,-94.584701,http://curiocollection3.hilton.com/en/hotels/m...
4,20,The Inn at Solvang,"Bed Breakfasts,Bed Breakfast,Hotels Motels,Hotel",Accommodation & Food Services,10611 Standing Stone Rd,Huntingdon,PA,40.527478,-77.969763,http://www.solvang.com


## Add hotel characteristics

### Distance from airport

In [3]:
# Get latitudes and longitudes of airports mapped in the USA
# Source: https://opendata.socrata.com/dataset/Airport-Codes-mapped-to-Latitude-Longitude-in-the-/rxrh-4cxm
path = "Data/Airport_Codes_Coords_USA.csv"

airports = pd.read_csv(path)
airports.head()

,locationID,Latitude,Longitude
0,ADK,51.8781,176.6461
1,AKK,56.9386,154.1825
2,Z13,60.9047,161.4225
3,AKI,60.9028,161.2306
4,AUK,62.6800,164.6600


In [4]:
print(f"Number of airports mapped in USA: {airports.shape[0]}")

Number of airports mapped in USA: 13429


In [5]:
# Haversine formula to calculate distance
# Source1: https://stackoverflow.com/a/41337005
# Source2: https://stackoverflow.com/a/21623206
from math import cos, asin, sqrt

def distance(lat1, lon1, lat2, lon2):
    """distance is expressed in km"""
    p = 0.017453292519943295 # pi/180; factor to convert degrees to radians
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return 12742 * asin(sqrt(a)) # Earth diameter: 12742 = 2 * R; R = 6371km (mean radius of the earth)

# Find the minimum distance between the hotel and the closest airport
def min_distance(lat, lon):
    distances = []
    for i in range(0, len(airports)):
        away = distance(abs(lat), abs(lon), abs(airports["Latitude"][i]), abs(airports["Longitude"][i]))
        distances.append(away)
    return min(distances)

In [6]:
# For each hotel coordinate, calculate the distance to the nearest airport
airport_distance = []
for i in range(0, len(metadata)):
    dist = min_distance(metadata["latitude"][i], metadata["longitude"][i])
    airport_distance.append(dist)
    
    print(f"Now processing {i}th airport.\n-----")
    clear_output(wait = True) # to replace output with new one (instead of printing many outputs)

Now processing 1852th airport.
-----


In [7]:
# Add airport distance in metadata
metadata["airportDistance_km"] = airport_distance

# Preview the dataframe
metadata.head()

,index,name,categories,primaryCategories,address,city,province,latitude,longitude,websites,airportDistance_km
0,0,Rancho Valencia Resort Spa,"Hotels,Hotels and motels,Hotel and motel reser...",Accommodation & Food Services,5921 Valencia Cir,Rancho Santa Fe,CA,32.990959,-117.186136,http://www.ranchovalencia.com,14.308848
1,3,Aloft Arundel Mills,"Hotels,Hotels and motels,Travel agencies and b...",Accommodation & Food Services,7520 Teague Rd,Hanover,MD,39.155929,-76.716341,http://www.starwoodhotels.com/alofthotels/prop...,4.668332
2,9,Hampton Inn Suites PortlandVancouver,"Hotels,Hotels and motels,Hotel and motel reser...",Accommodation & Food Services,315 SE Olympia Dr,Vancouver,WA,45.619212,-122.525196,http://hamptoninn3.hilton.com/en/hotels/washin...,6.591900
3,15,Hotel Phillips,"Hotels,Caterers,Hotels and motels,Hotel,Restau...",Accommodation & Food Services,106 W 12th St,Kansas City,MO,39.100119,-94.584701,http://curiocollection3.hilton.com/en/hotels/m...,2.670645
4,20,The Inn at Solvang,"Bed Breakfasts,Bed Breakfast,Hotels Motels,Hotel",Accommodation & Food Services,10611 Standing Stone Rd,Huntingdon,PA,40.527478,-77.969763,http://www.solvang.com,3.781817


### Hotel features

In [8]:
# Tokenise the categories and put in a new column
cats = []
for row in metadata["categories"]:
    tokens = word_tokenize(row)
    cats.append(tokens)

cats2 = []
for cat in cats:
    x = []
    for word in cat:
        x.append(word.lower())
    cats2.append(x)
metadata["categories2"] = cats2

In [9]:
metadata.head()

,index,name,categories,primaryCategories,address,city,province,latitude,longitude,websites,airportDistance_km,categories2
0,0,Rancho Valencia Resort Spa,"Hotels,Hotels and motels,Hotel and motel reser...",Accommodation & Food Services,5921 Valencia Cir,Rancho Santa Fe,CA,32.990959,-117.186136,http://www.ranchovalencia.com,14.308848,"[hotels, ,, hotels, and, motels, ,, hotel, and..."
1,3,Aloft Arundel Mills,"Hotels,Hotels and motels,Travel agencies and b...",Accommodation & Food Services,7520 Teague Rd,Hanover,MD,39.155929,-76.716341,http://www.starwoodhotels.com/alofthotels/prop...,4.668332,"[hotels, ,, hotels, and, motels, ,, travel, ag..."
2,9,Hampton Inn Suites PortlandVancouver,"Hotels,Hotels and motels,Hotel and motel reser...",Accommodation & Food Services,315 SE Olympia Dr,Vancouver,WA,45.619212,-122.525196,http://hamptoninn3.hilton.com/en/hotels/washin...,6.591900,"[hotels, ,, hotels, and, motels, ,, hotel, and..."
3,15,Hotel Phillips,"Hotels,Caterers,Hotels and motels,Hotel,Restau...",Accommodation & Food Services,106 W 12th St,Kansas City,MO,39.100119,-94.584701,http://curiocollection3.hilton.com/en/hotels/m...,2.670645,"[hotels, ,, caterers, ,, hotels, and, motels, ..."
4,20,The Inn at Solvang,"Bed Breakfasts,Bed Breakfast,Hotels Motels,Hotel",Accommodation & Food Services,10611 Standing Stone Rd,Huntingdon,PA,40.527478,-77.969763,http://www.solvang.com,3.781817,"[bed, breakfasts, ,, bed, breakfast, ,, hotels..."


In [10]:
# Create a list of unique words to chose features from
word_list = []
for row in metadata["categories2"]:
    for word in row:
        word_list.append(word)
        
unique_terms = list(set(word_list))
unique_terms

['courses',
 'trade',
 'arrangers',
 'mid',
 'lodge',
 'extended',
 'pools',
 'convention',
 'diego',
 'springs',
 'harbor',
 'alternative',
 'packages',
 'titusville',
 'concierge',
 'tourist',
 'casinos',
 'side',
 'clearwater',
 'lanham-seabrook',
 'perimeter',
 'restaurants',
 'lower',
 'boutique',
 'therapists',
 'spa',
 'centers',
 'fitness',
 'slides',
 'services',
 'casino',
 'northeast',
 'grill',
 'mediterranean',
 'suites',
 'stonelake',
 'golf',
 'arlington',
 'stay',
 'downtown',
 'bed',
 'theater',
 'ski',
 'spot',
 'italian',
 'skiing',
 'night',
 'family-friendly',
 'moultonborough',
 'woodbridge',
 'movie',
 'philadelphia',
 'vacation',
 'nightlife',
 'care',
 'real',
 'conventions',
 'garden',
 'internet',
 "d'alene",
 'fullerton',
 'halls',
 'airports',
 'rentals',
 'creek',
 'mountain',
 'brunch',
 'consultants',
 'new',
 'pennsylvania',
 'marinas',
 'safety',
 'chalets',
 'dining',
 'gym',
 'mexican',
 'southwest',
 'park',
 'corner',
 'home',
 'uptown-galleria',
 

In [11]:
# Create new columns containing categories/features for the hotels (manually pick from unique terms)
features = ['resort', 'movie', 'marina', 'harbor', 'reception',
           'convention', 'health', 'hall', 'extended', 'lodges',
           'chalets', 'cemetery', 'skiing', 'theater', 'campground',
           'entertainment', 'clinics', 'cabins', 'parties', 'lodge', 
           'nightclub', 'services', 'airport', 'spas', 'hotel', 'pools',
           'attractions', 'meeting', 'utility', 'condominiums', 'cable',
           'office', 'village', 'fashion', 'loft', 'chapels', 'fairgrounds', 
           'boutique', 'gym', 'motel', 'bars', 'e-commerce',
           'golf', 'apartment', 'medical', 'pubs', 'cottages', 'pet', 'lakeview',
           'restaurant', 'wedding', 'fitness', 'recreation', 'receptions', 
           'reservations', 'casino', 'family-friendly', 'breakfast',
           'beach', 'karaoke']

# Sort the items in the list alphabetically
features.sort()

for feat in features:
    metadata[feat] = np.nan

In [12]:
# Create a function that fills in 1s and 0s for selected categories
def Cat_encoding(category):
    for i in range(0,len(metadata)):
        if category not in metadata["categories2"][i]:
            metadata[str(category)][i] = "0"
        else:
            metadata[str(category)][i] = "1" 
        
        print(f"Now processing {i}th hotel for {category}.\n-----")
        clear_output(wait = True) # to replace output with new one (instead of printing many outputs)  

In [13]:
# Loop the Cat_encoding function to fill the empty categories
for x in features:
    Cat_encoding(x) 

Now processing 1852th hotel for wedding.
-----


In [14]:
# Create a list of unique provinces
prov_list = list(set(metadata["province"]))

# Sort the items in the list alphabetically
prov_list.sort()

for prov in prov_list:
    metadata[prov] = np.nan

In [15]:
# Create a function that fills in 1s and 0s for selected categories
def Prov_encoding(province):
    for i in range(0,len(metadata)):
        if province not in metadata["province"][i]:
            metadata[str(province)][i] = "0"
        else:
            metadata[str(province)][i] = "1" 
        
        print(f"Now processing {i}th hotel for {province}.\n-----")
        clear_output(wait = True) # to replace output with new one (instead of printing many outputs)  

In [16]:
# Loop the Prov_encoding function to fill the empty provinces
for x in prov_list:
    Prov_encoding(x) 

KeyboardInterrupt: 

In [ ]:
metadata.head()

## Load the metadata with new information into a new database table 

In [ ]:
# Create a list of columns
columns = list(metadata.columns)

In [ ]:
# Create a function that converts the values into strings
def to_str(column_name):
    metadata[column_name] = metadata[column_name].astype(str)

In [ ]:
# Loop through all columns and convert their values into strings
for column in columns:
    to_str(column)

# Check the data types for each column    
metadata.dtypes    

In [ ]:
# Drop columns from metadata df
metadata = metadata.drop(columns = ["categories", "primaryCategories", "address", "latitude",
                                    "longitude", "websites", "categories2", "city"])
metadata.head()

In [ ]:
# Create a connection to the database
conn = sqlite3.connect("Data/Hotels.db")

# Save the dataframe as a sqlite database table
metadata.to_sql("metadata2", conn, if_exists = "replace", index = False)

In [ ]:
# Preview metadata
pd.read_sql_query("select * from metadata2 limit 3;", conn)

In [ ]:
# Close the connection
conn.close()